In [1]:
%%time
## 1
############################1
import pandas as pd
import numpy as np
import sys
import random
import time
import warnings
from file_paths_and_consts import *
import math
import s2sphere
import resource


warnings.filterwarnings('ignore')

random.seed(time.time())

start_time = time.time()


#############################2
APPLY_PEER = 1
EPS = 0.0001 #0.0001 ## calibrated

PLACE_NAME = 'Poltavskyi'
print(PLACE_NAME)
hyper_comb = 99999

DIS_EXPONENT = 2.212
A = 55.0 
T = 0.253 
S = 98.67 
lookbefore_days_left = 18
lookbefore_days_right = 7
PROB_SCALAR = 0.3509143396912264
EVENT_WEIGHT_SCALAR = 0.35229225061104735
USE_PEER_EFFECT = 40
USE_CIVIL_DATA = 0
THRESH_LO = 2
THRESH_HI = 15
STRUCT = 13
CHANGE_NETWORK = 1
USE_NEIGHBOR = 5
BORDER_CROSS_PROB = 0.25676486858191583
PHASE_SHIFT = 40

ablation_conflict_type = 'None'

MOVE_PROB = [0.25,0.7,0.02,0.7]
FAMILY_PROB = [0.25,0.85,0.1,0.85]

multiply_lo = random.uniform(0.95,1.0)#0.8-0.9
multiply_hi = random.uniform(1.5,2.0)
multiply_very_hi = random.uniform(2.5,2.8)

for i in range(len(MOVE_PROB)):
    MOVE_PROB[i] = MOVE_PROB[i]*PROB_SCALAR
    FAMILY_PROB[i] = FAMILY_PROB[i]*PROB_SCALAR

##################################3
def haversine(lon1, lat1, lon2, lat2):
    KM = 6372.8 #Radius of earth in km instead of miles
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    total_km = KM * c
    return total_km

## rule about how each agent is affected by each impact
def prob_conflict(impact,dis,t_diff=0):
    return ((impact)/(dis**DIS_EXPONENT))*(1.0/(1+t_diff))

## rule about how an agent is affected overall by all impact   (P(violence))
def aggregated_prob_conflict(x):  
    return 1 / (1 + A*math.exp(-T*x))


#https://www.nature.com/articles/s41599-018-0094-8
def memory_decay(x):
    return x*(S/100.00)

## rule about how different types of demographic group decides to move
def get_move_prob(DEMO_NUMS): ##this function can be played with
    tot_size = 0
    for v in DEMO_NUMS:
        tot_size = tot_size+v
    if tot_size>1:
        move_prob = 0.0
        for i in range(0,len(DEMO_NUMS)):
            move_prob = move_prob + DEMO_NUMS[i]*FAMILY_PROB[i]
        return move_prob/tot_size
    else:
        move_prob = 0.0
        for i in range(0,len(DEMO_NUMS)):
            move_prob = move_prob + DEMO_NUMS[i]*MOVE_PROB[i]
        return move_prob/tot_size

def bernoulli(val,p):
    if (val<=p):
        return 1
    else:
        return 0
    
def bernoulli_border(val,moves,current_phase):
    if current_phase == 0:
        multiply = multiply_lo
    elif current_phase==1:
        multiply = multiply_hi
    else:
        multiply = multiply_hi
    if moves==0:
        return 0
    else:
        if val<=BORDER_CROSS_PROB*multiply:
            return 2
        else:
            return 1
    
def getl13(lat,lng,req_level=13):
    p = s2sphere.LatLng.from_degrees(lat, lng) 
    cell = s2sphere.Cell.from_lat_lng(p)
    cellid = cell.id()
    for i in range(1,30):
        #print(cellid)
        if cellid.level()==req_level:
            return cellid
        cellid = cellid.parent()

        
## thresh_type==0
## if my current decision is to migrate (1), if less than thresh_lo of my neighbors are migrating, i will change my decision to (0)

## thresh_type==1
## if my current decision is not to migrate(0), if at least thresh_hi of my neigbhors are migrating, i will change my decision to (1)

def apply_threshold_function(cur_decision,cur_neighbor_migrating,thresh_lo=THRESH_LO,thresh_hi=THRESH_HI):
        if cur_decision==1:
            if cur_neighbor_migrating<thresh_lo:
                return 0
        if cur_decision==0:
            if cur_neighbor_migrating>=thresh_hi:
                return 1
        return cur_decision
    
def refine_through_peer_effect_old(temp_household,current_phase):
    if current_phase==0:
        cur_struct = STRUCT
    else:
        cur_struct = STRUCT-1
    if CHANGE_NETWORK==1:
        cur_struct = STRUCT
    
    new_temp_households = temp_household.sort_values(by='hid')
    new_temp_households['s2_cell'] = new_temp_households.apply(lambda x: getl13(x['h_lat'],x['h_lng'],cur_struct),axis=1)
    h_decision = new_temp_households[['hid','s2_cell','moves']]

    h_network = h_decision.merge(h_decision,on='s2_cell',how='inner')
    #h_network = h_network[h_network.hid_x!=h_network.hid_y]
    h_network_peer_move_count = (h_network.groupby('hid_x')['moves_y'].sum().reset_index()).merge(h_network[['hid_x','moves_x']].drop_duplicates(),on='hid_x',how='inner')
    h_network_peer_move_count['moves_y'] = h_network_peer_move_count['moves_y'] - h_network_peer_move_count['moves_x']
    h_network_peer_move_count['peer_affected_move'] = h_network_peer_move_count.apply(lambda x: apply_threshold_function(x['moves_x'],x['moves_y']),axis=1)
    h_network_peer_move_count = h_network_peer_move_count.sort_values(by='hid_x')

    #print(h_network_peer_move_count[h_network_peer_move_count.moves_x==1].shape,h_network_peer_move_count[h_network_peer_move_count.peer_affected_move==1].shape)
    
    new_temp_households['moves'] = h_network_peer_move_count['peer_affected_move']
    return new_temp_households

def refine_through_peer_effect(temp_household,current_phase=0):
    
    new_temp_households = temp_household.sort_values(by='hid')
    
    network_creation_start = time.time()
    cur_neighbor_household = neighbor_household_data[neighbor_household_data.hid_x.isin(new_temp_households.hid.tolist())]
    cur_neighbor_household = cur_neighbor_household[cur_neighbor_household.hid_y.isin(new_temp_households.hid.tolist())]
    cur_neighbor_household = cur_neighbor_household.merge(new_temp_households[['hid','moves']],left_on='hid_x',right_on='hid',how='inner')
    cur_neighbor_household = cur_neighbor_household.rename(columns={'moves':'moves_x'})
    cur_neighbor_household = cur_neighbor_household.drop(columns=['hid'])
    h_network = cur_neighbor_household.merge(new_temp_households[['hid','moves']],left_on='hid_y',right_on='hid',how='inner')
    h_network = h_network.rename(columns={'moves':'moves_y'})
    h_network = h_network.drop(columns=['hid'])
    print('network creation through node merging by s2 takes',time.time()-network_creation_start,'seconds')
    
    neighbor_count_start = time.time()
    h_network_peer_move_count = (h_network.groupby('hid_x')['moves_y'].sum().reset_index()).merge(h_network[['hid_x','moves_x']].drop_duplicates(),on='hid_x',how='inner')
    print('neighbor countint through groupby takes',time.time()-neighbor_count_start,'seconds')
    
    h_network_peer_move_count['moves_y'] = h_network_peer_move_count['moves_y'] - h_network_peer_move_count['moves_x']
    threshold_func_start = time.time()
    h_network_peer_move_count['peer_affected_move'] = h_network_peer_move_count.apply(lambda x: apply_threshold_function(x['moves_x'],x['moves_y']),axis=1)
    print('applying threshold function takes',time.time()-threshold_func_start,'seconds')
    h_network_peer_move_count = h_network_peer_move_count.sort_values(by='hid_x')

    #print(h_network_peer_move_count[h_network_peer_move_count.moves_x==1].shape,h_network_peer_move_count[h_network_peer_move_count.peer_affected_move==1].shape)
    print(h_network_peer_move_count.shape,new_temp_households.shape)
    new_temp_households['moves'] = h_network_peer_move_count['peer_affected_move']
    return new_temp_households

def get_event_weight(event_type,sub_event_type):
    if sub_event_type==ablation_conflict_type:
        return 0
    if event_type=="Battles":
        return 3
    if event_type.startswith('Civilian'):
        return 8
    if event_type.startswith('Explosions'):
        return 5
    if event_type.startswith('Violence'):
        return 3
    if event_type.startswith('Protests') or event_type.startswith('Riots'):
        return 0
    return 0

#####################################4
neighbor_data = pd.read_csv(UNCLEANED_DATA_DIR+'neighbor_raions.csv')
neighbor_list = neighbor_data[neighbor_data.ADM2_EN_x==PLACE_NAME]['ADM2_EN_y'].unique().tolist()

if USE_NEIGHBOR==0:
    impact_data = pd.read_csv(IMPACT_DIR+'ukraine_conflict_data_ADM2_HDX.csv')
else:
    impact_data = pd.read_csv(IMPACT_DIR+'ukraine_conflict_data_ADM2_HDX_buffer_'+str(USE_NEIGHBOR)+'_km.csv')

impact_data['time'] = pd.to_datetime(impact_data['time'])

if USE_CIVIL_DATA==1:
    civilian_data = pd.read_csv(IMPACT_DIR+'ukraine_civilian_conflict_data_ADM2_HDX.csv')
    civilian_data['time'] = pd.to_datetime(civilian_data['time'])

    impact_data = pd.concat([impact_data,civilian_data])

if USE_NEIGHBOR==1:
    impact_data = impact_data[impact_data.matching_place_id.isin(neighbor_list)]
    impact_data = impact_data.drop(columns=['matching_place_id'])
    
cur_household_data = pd.read_csv(HOUSEHOLD_DIR+'ukraine_household_data_ADM2_HDX.csv')
refugee_data = pd.read_csv(GROUND_TRUTH_DIR+'ukraine_refugee_data_2.csv')
refugee_data['time'] = pd.to_datetime(refugee_data['time'])


impact_data['event_weight'] = impact_data.apply(lambda x: get_event_weight(x['event_type'],x['sub_event_type']),axis=1)
print('impact events reassigned',impact_data.shape)

cur_household_data = cur_household_data[cur_household_data.matching_place_id==PLACE_NAME]
cur_household_data['hh_size'] = cur_household_data[DEMO_TYPES].sum(axis = 1, skipna = True)
cur_household_data['P(move|violence)'] = cur_household_data.apply(lambda x: get_move_prob([x['OLD_PERSON'],x['CHILD'],x['ADULT_MALE'],x['ADULT_FEMALE']]),axis=1)
cur_household_data['prob_conflict'] = 0
cur_household_data['moves'] = 0
cur_household_data['move_type'] = 0 # 0 means did not move, 1 means IDP, 2 means outside

if 'h_lat' not in cur_household_data.columns.tolist():
    cur_household_data = cur_household_data.rename(columns={'latitude':'h_lat','longitude':'h_lng'})

print('household properties updated')
print('this oblast size',cur_household_data.shape)

neighbor_household_data = pd.read_csv(HOUSEHOLD_DIR+'ukraine_neighbor_'+PLACE_NAME+'_'+str(STRUCT)+'_s2.csv')

min_date = pd.to_datetime('2022-02-24')
end_date = pd.to_datetime('2022-05-15')
simulated_refugee_df = pd.DataFrame(columns=['id','time','refugee','old_people','child','male','female'])
simulated_leaving_df = pd.DataFrame(columns=['id','time','leaving','old_people','child','male','female'])
timing_log_df = pd.DataFrame(columns=['step','remaining_household_agent','remaining_person_agent','conflict_events_now','network_nodes',
                                      'network_edges','attitude_time','pcb_time','subjective_norm_time','pre_time','post_time'])

temp_prefix = ''
if ablation_conflict_type!='None':
    OUTPUT_DIR = ABLATION_DIR
    temp_prefix = 'ablation_'
    
print(OUTPUT_DIR)
print(temp_prefix)

f = 0

start = time.time()
cur_checkpoint = 1000
print('combination_no',hyper_comb)

prev_temp_checkpoint = 0
last_saved_checkpoint = -1

peer_used = 0

DEL_COLUMNS = ['P(violence)','P(move)','random']

who_went_where = []
hid_displacement_df = []

Poltavskyi
impact events reassigned (34022, 10)
household properties updated
this oblast size (274572, 20)
/project/biocomplexity/UKR_forecast/migration_data/output_data/

combination_no 99999
CPU times: user 48.6 s, sys: 3.87 s, total: 52.5 s
Wall time: 53.4 s


In [5]:
#########################################5
for step in range(0,1):
    print(i)
    preprocess_start = time.time()

    prev_temp_checkpoint = prev_temp_checkpoint + 1

    max_date = min_date + pd.DateOffset(days=1)

    lookahead_date_1 = min_date - pd.DateOffset(days=lookbefore_days_left)
    lookahead_date_2 = min_date - pd.DateOffset(days=lookbefore_days_right)

    if(f==1 and min_date > end_date):
        break
    ##################
    print(min_date)
    if(f!=0):
        cur_household_data = pd.read_csv(TEMPORARY_DIR+'last_saved_household_data_'+str(temp_prefix)+str(PLACE_NAME)+'_'+str(hyper_comb)+'.csv')
        cur_household_data = cur_household_data[cur_household_data.moves==0]

    ##################

    cur_impact_data = impact_data[(impact_data.time>=lookahead_date_1) & (impact_data.time<=lookahead_date_2)]
    #cur_impact_data = cur_conflict_data[cur_conflict_columns]
    preprocess_end = time.time()

    attitude_start = time.time()
    cur_impact_data = cur_impact_data.rename(columns={'latitude':'impact_lat','longitude':'impact_lng'})
    cur_impact_data['cur_time'] = min_date
    cur_impact_data['time_diff_to_event'] = (cur_impact_data['cur_time'] - cur_impact_data['time']) / np.timedelta64(1,'D')
    cur_impact_data['impact_intensity'] = cur_impact_data['event_weight']*cur_impact_data['event_intensity']*EVENT_WEIGHT_SCALAR
    cur_impact_data['impact_intensity'].replace(to_replace = 0, value = EPS, inplace=True)

    if(cur_impact_data.shape[0]==0):
        new_row = {'id':PLACE_NAME,'time':min_date,'refugee':0,'old_people':0,'child':0,'male':0,'female':0}
        new_row_2 = {'id':PLACE_NAME,'time':min_date,'leaving':0,'old_people':0,'child':0,'male':0,'female':0}
        min_date = max_date
        simulated_refugee_df = simulated_refugee_df.append(new_row,ignore_index=True)
        simulated_leaving_df = simulated_leaving_df.append(new_row_2,ignore_index=True)
        continue

    
    impact_in_homes = cur_impact_data.merge(cur_household_data,on='matching_place_id',how='inner')
    impacted_household_list = impact_in_homes.hid.unique().tolist()
    household_not_impacted = cur_household_data[~cur_household_data.hid.isin(impacted_household_list)]

    if impact_in_homes.shape[0]==0:
        new_row = {'id':PLACE_NAME,'time':min_date,'refugee':0,'old_people':0,'child':0,'male':0,'female':0}
        new_row_2 = {'id':PLACE_NAME,'time':min_date,'leaving':0,'old_people':0,'child':0,'male':0,'female':0}
        min_date = max_date
        simulated_refugee_df = simulated_refugee_df.append(new_row,ignore_index=True)
        simulated_leaving_df = simulated_leaving_df.append(new_row_2,ignore_index=True)
        continue
    #print(ukraine_conflict_in_homes['fatalities'].describe())
    impact_in_homes['dis_conflict_home'] = haversine(impact_in_homes['h_lng'],impact_in_homes['h_lat'],impact_in_homes['impact_lng'],impact_in_homes['impact_lat'])
    impact_in_homes['prob_conflict'] = impact_in_homes['prob_conflict'].apply(lambda x: memory_decay(x))
    impact_in_homes['prob_conflict'] = impact_in_homes['prob_conflict'] + impact_in_homes.apply(lambda x: prob_conflict(x['impact_intensity'],x['dis_conflict_home'],x['time_diff_to_event']),axis=1)

    cur_household_data = cur_household_data.drop(columns='prob_conflict')
    home_conflict_df = impact_in_homes.groupby(['hid'])['prob_conflict'].sum().reset_index()
    home_conflict_df['P(violence)'] = home_conflict_df.prob_conflict.apply(aggregated_prob_conflict)
    home_conflict_df = home_conflict_df.merge(cur_household_data,on='hid',how='inner')
    #print('number of households with p(violence)',len(home_conflict_df.hid.unique().tolist()),home_conflict_df.shape)
    attitude_end = time.time()

    pcb_start = time.time()
    home_conflict_df['P(move)'] = home_conflict_df['P(violence)']*home_conflict_df['P(move|violence)']
    home_conflict_df['random'] = np.random.random(home_conflict_df.shape[0])
    home_conflict_df['moves'] = home_conflict_df.apply(lambda x: bernoulli(x['random'],x['P(move)']),axis=1)
    #print('number of households moving or not moving',home_conflict_df.shape)



    home_conflict_df = home_conflict_df.drop(columns=DEL_COLUMNS)

    print('###########################')
    pcb_end = time.time()

    subjective_norm_start = time.time()
    temp_households = pd.concat([home_conflict_df,household_not_impacted])
    nodes = temp_households.shape[0]

    #border_cross_low = 1 if peer_used<USE_PEER_EFFECT else 0
    phase = 1 
    #if(peer_used<min(30,USE_PEER_EFFECT)):
    #    border_cross_low = 2
    if(peer_used<PHASE_SHIFT):
        phase = 0

    if APPLY_PEER==1 and peer_used<USE_PEER_EFFECT:
        temp_households = refine_through_peer_effect(temp_households,phase)
        peer_used = peer_used + 1

    temp_households['move_type_random'] = np.random.random(temp_households.shape[0])
    temp_households['move_type'] = temp_households.apply(lambda x: bernoulli_border(x['move_type_random'],x['moves'],phase),axis=1)
    temp_households = temp_households.drop(columns=['move_type_random'])
    subjective_norm_end = time.time()

    post_process_and_save_start = time.time()

    new_row = {'id':PLACE_NAME,'time':min_date,'refugee':temp_households[temp_households.move_type==2]['hh_size'].sum(),
               'old_people':temp_households[temp_households.move_type==2]['OLD_PERSON'].sum(),'child':temp_households[temp_households.move_type==2]['CHILD'].sum(),
                'male':temp_households[temp_households.move_type==2]['ADULT_MALE'].sum(),'female':temp_households[temp_households.move_type==2]['ADULT_FEMALE'].sum()}
    simulated_refugee_df = simulated_refugee_df.append(new_row,ignore_index=True)

    new_row_2 = {'id':PLACE_NAME,'time':min_date,'leaving':temp_households[temp_households.moves==1]['hh_size'].sum(),
               'old_people':temp_households[temp_households.moves==1]['OLD_PERSON'].sum(),'child':temp_households[temp_households.moves==1]['CHILD'].sum(),
                'male':temp_households[temp_households.moves==1]['ADULT_MALE'].sum(),'female':temp_households[temp_households.moves==1]['ADULT_FEMALE'].sum()}

    simulated_leaving_df = simulated_leaving_df.append(new_row_2,ignore_index=True)

    temp_households['move_date'] = str(min_date)
    hid_displacement_df.append(temp_households[temp_households.move_type!=0])
    temp_households = temp_households.drop(columns=['move_date'])

    curtime = time.time()
    if((curtime-start)>=cur_checkpoint):
        print('checkpoint for ',str(PLACE_NAME))
        simulated_refugee_df.to_csv(OUTPUT_DIR+'mim_result_'+str(PLACE_NAME)+'_'+str(hyper_comb).zfill(5)+'.csv',index=False)
        simulated_leaving_df.to_csv(OUTPUT_DIR+'mim_result_leaving_'+str(PLACE_NAME)+'_'+str(hyper_comb).zfill(5)+'.csv',index=False)
        start = curtime

    temp_households.to_csv(TEMPORARY_DIR+'last_saved_household_data_'+str(temp_prefix)+str(PLACE_NAME)+'_'+str(hyper_comb)+'.csv',index=False)

    print('intention module done')

    ##apply destination module from here
    moving_households_df = temp_households[temp_households.move_type==2].reset_index()
    dest_prob_distribution_df = pd.read_csv(OUTPUT_DIR+'destination_probability_distribution.csv')

    if(str(min_date) not in dest_prob_distribution_df.columns.tolist()):
        cur_dest_prob_df = dest_prob_distribution_df[['ISO','2022-03-01 00:00:00']]
        weight_col = '2022-03-01 00:00:00'
    else:
        cur_dest_prob_df = dest_prob_distribution_df[['ISO',str(min_date)]]
        weight_col = str(min_date)

    dest_sampled = cur_dest_prob_df.sample(moving_households_df.shape[0],replace=True,weights=cur_dest_prob_df[weight_col]).reset_index()

    moving_households_df['destination'] = dest_sampled['ISO']
    moving_households_df['moved_date'] = str(min_date)

    who_went_where.append(moving_households_df)

    print('destination module done')

    last_saved_checkpoint = prev_temp_checkpoint
    min_date = max_date
    f = 1

    post_process_and_save_end = time.time()

    timing_row = {'step':i,'remaining_household_agent':cur_household_data.shape[0],'remaining_person_agent':cur_household_data['hh_size'].sum(),'conflict_events_now':cur_impact_data.shape[0],
                  'network_nodes':nodes,'network_edges':nodes*nodes,'attitude_time':attitude_end-attitude_start,'pcb_time':pcb_end-pcb_start,
                  'subjective_norm_time':subjective_norm_end-subjective_norm_start,'pre_time':preprocess_end-preprocess_start,'post_time':post_process_and_save_end-post_process_and_save_start}
    timing_log_df = timing_log_df.append(timing_row,ignore_index=True)
    i = i + 1
    


#print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)

3
2022-02-27 00:00:00
###########################
network creation through node merging by s2 takes 44.381754636764526 seconds
neighbor countint through groupby takes 4.48286247253418 seconds
applying threshold function takes 2.339191436767578 seconds
(274572, 4) (274572, 20)
checkpoint for  Poltavskyi
intention module done
destination module done


In [8]:
cur_impact_data.head()

,event_id,time,impact_lat,impact_lng,event_type,sub_event_type,event_weight,event_intensity,matching_place_name,matching_place_id,cur_time,time_diff_to_event,impact_intensity
537,8867872,2022-02-20,48.6630,39.3098,Explosions/Remote violence,Shelling_or_artillery_or_missile_attack,5,0,Luhanskyi,Luhanskyi,2022-02-27,7.0,0.0001
538,8867873,2022-02-20,48.6849,39.1812,Explosions/Remote violence,Shelling_or_artillery_or_missile_attack,5,0,Luhanskyi,Luhanskyi,2022-02-27,7.0,0.0001
539,8868303,2022-02-20,48.7895,39.3084,Explosions/Remote violence,Shelling_or_artillery_or_missile_attack,5,0,Shchastynskyi,Luhanskyi,2022-02-27,7.0,0.0001
540,8868341,2022-02-20,48.6543,39.4794,Explosions/Remote violence,Shelling_or_artillery_or_missile_attack,5,0,Shchastynskyi,Luhanskyi,2022-02-27,7.0,0.0001
541,8868346,2022-02-20,48.7272,39.1455,Explosions/Remote violence,Shelling_or_artillery_or_missile_attack,5,0,Shchastynskyi,Luhanskyi,2022-02-27,7.0,0.0001


In [9]:
impact_in_homes.head()

,event_id,time,impact_lat,impact_lng,event_type,sub_event_type,event_weight,event_intensity,matching_place_name,matching_place_id,...,ADM1_EN,ADM1_PCODE,admin1,admin2,hh_size,P(move|violence),prob_conflict,moves,move_type,dis_conflict_home
0,8868580,2022-02-20,49.5937,34.5407,Protests,Peaceful_protest,0,0,Poltavskyi,Poltavskyi,...,Poltavska,UA53,140003823,160024564,1,0.007018,1.081440e-09,0,0,68.668495
1,8868580,2022-02-20,49.5937,34.5407,Protests,Peaceful_protest,0,0,Poltavskyi,Poltavskyi,...,Poltavska,UA53,140003823,160024572,1,0.087729,3.649709e-09,0,0,39.622679
2,8868580,2022-02-20,49.5937,34.5407,Protests,Peaceful_protest,0,0,Poltavskyi,Poltavskyi,...,Poltavska,UA53,140003823,160024576,1,0.087729,5.699294e-09,0,0,32.392020
3,8868580,2022-02-20,49.5937,34.5407,Protests,Peaceful_protest,0,0,Poltavskyi,Poltavskyi,...,Poltavska,UA53,140003823,160024576,1,0.007018,3.131836e-09,0,0,42.460854
4,8868580,2022-02-20,49.5937,34.5407,Protests,Peaceful_protest,0,0,Poltavskyi,Poltavskyi,...,Poltavska,UA53,140003823,160024577,1,0.245640,1.409525e-08,0,0,21.510817


In [10]:
home_conflict_df.head()

,hid,prob_conflict,OLD_PERSON,CHILD,ADULT_MALE,ADULT_FEMALE,rlid,h_lat,h_lng,ADM2_EN,ADM2_PCODE,ADM1_EN,ADM1_PCODE,admin1,admin2,matching_place_id,hh_size,P(move|violence),moves,move_type
0,14,1.081440e-09,0,0,1,0,14,50.159608,34.157746,Poltavskyi,UA5308,Poltavska,UA53,140003823,160024564,Poltavskyi,1,0.007018,0,0
1,126,3.649709e-09,1,0,0,0,126,49.557404,35.087210,Poltavskyi,UA5308,Poltavska,UA53,140003823,160024572,Poltavskyi,1,0.087729,0,0
2,219,5.699294e-09,1,0,0,0,219,49.365947,34.261351,Poltavskyi,UA5308,Poltavska,UA53,140003823,160024576,Poltavskyi,1,0.087729,0,0
3,247,3.131836e-09,0,0,1,0,247,49.224157,34.687893,Poltavskyi,UA5308,Poltavska,UA53,140003823,160024576,Poltavskyi,1,0.007018,0,0
4,288,1.409525e-08,0,0,0,1,288,49.570690,34.836869,Poltavskyi,UA5308,Poltavska,UA53,140003823,160024577,Poltavskyi,1,0.245640,0,0


In [ ]:
##################################6
hid_all_displacement_df = pd.concat(hid_displacement_df)
hid_all_displacement_df.to_csv(OUTPUT_DIR+'mim_hid_completed_'+str(PLACE_NAME)+'_'+str(hyper_comb).zfill(5)+'.csv',index=False)

simulated_refugee_with_dest_df = pd.concat(who_went_where)

simulated_refugee_with_dest_df.to_csv(OUTPUT_DIR+'mdm_result_completed_'+str(PLACE_NAME)+"_"+str(hyper_comb).zfill(5)+'.csv',index=False)
simulated_refugee_df.to_csv(OUTPUT_DIR+'mim_result_completed_'+str(PLACE_NAME)+'_'+str(hyper_comb).zfill(5)+'.csv',index=False)
simulated_leaving_df.to_csv(OUTPUT_DIR+'mim_result_leaving_completed_'+str(PLACE_NAME)+'_'+str(hyper_comb).zfill(5)+'.csv',index=False)

end = time.time()
data = {'raion': [PLACE_NAME],'runtime':[end-start_time],'hyper_comb':[hyper_comb]}
run_df = pd.DataFrame(data)
 
# append data frame to CSV file
run_df.to_csv('../runtime_log/runtime_raion.csv', mode='a', index=False, header=False)
timing_log_df.to_csv(OUTPUT_DIR+'timing_log_'+str(PLACE_NAME)+'_'+str(hyper_comb).zfill(5)+'.csv',index=False)

#TODO:
#a) add timing module
#b) save info about remaining households and agents after each costly module, at least the number of households for now

In [6]:
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

122504

In [7]:
############################1
############################1
import pandas as pd
import numpy as np
import sys
import random
import time
import warnings
from file_paths_and_consts import *
import math
import s2sphere
import resource


warnings.filterwarnings('ignore')

random.seed(time.time())

start_time = time.time()


#############################2
APPLY_PEER = 1
EPS = 0.0001 #0.0001 ## calibrated

PLACE_NAME = sys.argv[1]
print(PLACE_NAME)
hyper_comb = int(sys.argv[2])

DIS_EXPONENT = float(sys.argv[3])
A = float(sys.argv[4]) 
T = float(sys.argv[5]) 
S = float(sys.argv[6]) 
lookbefore_days_left = int(sys.argv[7])
lookbefore_days_right = int(sys.argv[8])
PROB_SCALAR = float(sys.argv[9]) 
EVENT_WEIGHT_SCALAR = float(sys.argv[10])
USE_PEER_EFFECT = int(sys.argv[11])
USE_CIVIL_DATA = 0
THRESH_LO = int(sys.argv[12])
THRESH_HI = int(sys.argv[13])
STRUCT = int(sys.argv[14])
CHANGE_NETWORK = int(sys.argv[15])
USE_NEIGHBOR = int(sys.argv[16])
BORDER_CROSS_PROB = float(sys.argv[17])
PHASE_SHIFT = int(sys.argv[18])

ablation_conflict_type = 'None'

MOVE_PROB = [0.25,0.7,0.02,0.7]
FAMILY_PROB = [0.25,0.85,0.1,0.85]

multiply_lo = random.uniform(0.95,1.0)#0.8-0.9
multiply_hi = random.uniform(1.5,2.0)
multiply_very_hi = random.uniform(2.5,2.8)

for i in range(len(MOVE_PROB)):
    MOVE_PROB[i] = MOVE_PROB[i]*PROB_SCALAR
    FAMILY_PROB[i] = FAMILY_PROB[i]*PROB_SCALAR

##################################3
def haversine(lon1, lat1, lon2, lat2):
    KM = 6372.8 #Radius of earth in km instead of miles
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    total_km = KM * c
    return total_km

## rule about how each agent is affected by each impact
def prob_conflict(impact,dis,t_diff=0):
    return ((impact)/(dis**DIS_EXPONENT))*(1.0/(1+t_diff))

## rule about how an agent is affected overall by all impact   (P(violence))
def aggregated_prob_conflict(x):  
    return 1 / (1 + A*math.exp(-T*x))


#https://www.nature.com/articles/s41599-018-0094-8
def memory_decay(x):
    return x*(S/100.00)

## rule about how different types of demographic group decides to move
def get_move_prob(DEMO_NUMS): ##this function can be played with
    tot_size = 0
    for v in DEMO_NUMS:
        tot_size = tot_size+v
    if tot_size>1:
        move_prob = 0.0
        for i in range(0,len(DEMO_NUMS)):
            move_prob = move_prob + DEMO_NUMS[i]*FAMILY_PROB[i]
        return move_prob/tot_size
    else:
        move_prob = 0.0
        for i in range(0,len(DEMO_NUMS)):
            move_prob = move_prob + DEMO_NUMS[i]*MOVE_PROB[i]
        return move_prob/tot_size

def bernoulli(val,p):
    if (val<=p):
        return 1
    else:
        return 0
    
def bernoulli_border(val,moves,current_phase):
    if current_phase == 0:
        multiply = multiply_lo
    elif current_phase==1:
        multiply = multiply_hi
    else:
        multiply = multiply_hi
    if moves==0:
        return 0
    else:
        if val<=BORDER_CROSS_PROB*multiply:
            return 2
        else:
            return 1
    
def getl13(lat,lng,req_level=13):
    p = s2sphere.LatLng.from_degrees(lat, lng) 
    cell = s2sphere.Cell.from_lat_lng(p)
    cellid = cell.id()
    for i in range(1,30):
        #print(cellid)
        if cellid.level()==req_level:
            return cellid
        cellid = cellid.parent()

        
## thresh_type==0
## if my current decision is to migrate (1), if less than thresh_lo of my neighbors are migrating, i will change my decision to (0)

## thresh_type==1
## if my current decision is not to migrate(0), if at least thresh_hi of my neigbhors are migrating, i will change my decision to (1)

def apply_threshold_function(cur_decision,cur_neighbor_migrating,thresh_lo=THRESH_LO,thresh_hi=THRESH_HI):
        if cur_decision==1:
            if cur_neighbor_migrating<thresh_lo:
                return 0
        if cur_decision==0:
            if cur_neighbor_migrating>=thresh_hi:
                return 1
        return cur_decision
    
def refine_through_peer_effect_old(temp_household,current_phase):
    if current_phase==0:
        cur_struct = STRUCT
    else:
        cur_struct = STRUCT-1
    if CHANGE_NETWORK==1:
        cur_struct = STRUCT
    
    new_temp_households = temp_household.sort_values(by='hid')
    new_temp_households['s2_cell'] = new_temp_households.apply(lambda x: getl13(x['h_lat'],x['h_lng'],cur_struct),axis=1)
    h_decision = new_temp_households[['hid','s2_cell','moves']]

    h_network = h_decision.merge(h_decision,on='s2_cell',how='inner')
    #h_network = h_network[h_network.hid_x!=h_network.hid_y]
    h_network_peer_move_count = (h_network.groupby('hid_x')['moves_y'].sum().reset_index()).merge(h_network[['hid_x','moves_x']].drop_duplicates(),on='hid_x',how='inner')
    h_network_peer_move_count['moves_y'] = h_network_peer_move_count['moves_y'] - h_network_peer_move_count['moves_x']
    h_network_peer_move_count['peer_affected_move'] = h_network_peer_move_count.apply(lambda x: apply_threshold_function(x['moves_x'],x['moves_y']),axis=1)
    h_network_peer_move_count = h_network_peer_move_count.sort_values(by='hid_x')

    #print(h_network_peer_move_count[h_network_peer_move_count.moves_x==1].shape,h_network_peer_move_count[h_network_peer_move_count.peer_affected_move==1].shape)
    
    new_temp_households['moves'] = h_network_peer_move_count['peer_affected_move']
    return new_temp_households

def refine_through_peer_effect(temp_household,current_phase=0):
    
    new_temp_households = temp_household.sort_values(by='hid')
    
    network_creation_start = time.time()
    cur_neighbor_household = neighbor_household_data[neighbor_household_data.hid_x.isin(new_temp_households.hid.tolist())]
    cur_neighbor_household = cur_neighbor_household[cur_neighbor_household.hid_y.isin(new_temp_households.hid.tolist())]
    cur_neighbor_household = cur_neighbor_household.merge(new_temp_households[['hid','moves']],left_on='hid_x',right_on='hid',how='inner')
    cur_neighbor_household = cur_neighbor_household.rename(columns={'moves':'moves_x'})
    cur_neighbor_household = cur_neighbor_household.drop(columns=['hid'])
    h_network = cur_neighbor_household.merge(new_temp_households[['hid','moves']],left_on='hid_y',right_on='hid',how='inner')
    h_network = h_network.rename(columns={'moves':'moves_y'})
    h_network = h_network.drop(columns=['hid'])
    print('network creation through node merging by s2 takes',time.time()-network_creation_start,'seconds')
    
    neighbor_count_start = time.time()
    h_network_peer_move_count = (h_network.groupby('hid_x')['moves_y'].sum().reset_index()).merge(h_network[['hid_x','moves_x']].drop_duplicates(),on='hid_x',how='inner')
    print('neighbor countint through groupby takes',time.time()-neighbor_count_start,'seconds')
    
    h_network_peer_move_count['moves_y'] = h_network_peer_move_count['moves_y'] - h_network_peer_move_count['moves_x']
    threshold_func_start = time.time()
    h_network_peer_move_count['peer_affected_move'] = h_network_peer_move_count.apply(lambda x: apply_threshold_function(x['moves_x'],x['moves_y']),axis=1)
    print('applying threshold function takes',time.time()-threshold_func_start,'seconds')
    h_network_peer_move_count = h_network_peer_move_count.sort_values(by='hid_x')

    #print(h_network_peer_move_count[h_network_peer_move_count.moves_x==1].shape,h_network_peer_move_count[h_network_peer_move_count.peer_affected_move==1].shape)
    print(h_network_peer_move_count.shape,new_temp_households.shape)
    new_temp_households['moves'] = h_network_peer_move_count['peer_affected_move']
    return new_temp_households

def get_event_weight(event_type,sub_event_type):
    if sub_event_type==ablation_conflict_type:
        return 0
    if event_type=="Battles":
        return 3
    if event_type.startswith('Civilian'):
        return 8
    if event_type.startswith('Explosions'):
        return 5
    if event_type.startswith('Violence'):
        return 3
    if event_type.startswith('Protests') or event_type.startswith('Riots'):
        return 0
    return 0

#####################################4
neighbor_data = pd.read_csv(UNCLEANED_DATA_DIR+'neighbor_raions.csv')
neighbor_list = neighbor_data[neighbor_data.ADM2_EN_x==PLACE_NAME]['ADM2_EN_y'].unique().tolist()

if USE_NEIGHBOR==0:
    impact_data = pd.read_csv(IMPACT_DIR+'ukraine_conflict_data_ADM2_HDX.csv')
else:
    impact_data = pd.read_csv(IMPACT_DIR+'ukraine_conflict_data_ADM2_HDX_buffer_'+str(USE_NEIGHBOR)+'_km.csv')

impact_data['time'] = pd.to_datetime(impact_data['time'])

if USE_CIVIL_DATA==1:
    civilian_data = pd.read_csv(IMPACT_DIR+'ukraine_civilian_conflict_data_ADM2_HDX.csv')
    civilian_data['time'] = pd.to_datetime(civilian_data['time'])

    impact_data = pd.concat([impact_data,civilian_data])

if USE_NEIGHBOR==1:
    impact_data = impact_data[impact_data.matching_place_id.isin(neighbor_list)]
    impact_data = impact_data.drop(columns=['matching_place_id'])
    
cur_household_data = pd.read_csv(HOUSEHOLD_DIR+'ukraine_household_data_ADM2_HDX.csv')
refugee_data = pd.read_csv(GROUND_TRUTH_DIR+'ukraine_refugee_data_2.csv')
refugee_data['time'] = pd.to_datetime(refugee_data['time'])


impact_data['event_weight'] = impact_data.apply(lambda x: get_event_weight(x['event_type'],x['sub_event_type']),axis=1)
print('impact events reassigned',impact_data.shape)

cur_household_data = cur_household_data[cur_household_data.matching_place_id==PLACE_NAME]
cur_household_data['hh_size'] = cur_household_data[DEMO_TYPES].sum(axis = 1, skipna = True)
cur_household_data['P(move|violence)'] = cur_household_data.apply(lambda x: get_move_prob([x['OLD_PERSON'],x['CHILD'],x['ADULT_MALE'],x['ADULT_FEMALE']]),axis=1)
cur_household_data['prob_conflict'] = 0
cur_household_data['moves'] = 0
cur_household_data['move_type'] = 0 # 0 means did not move, 1 means IDP, 2 means outside

if 'h_lat' not in cur_household_data.columns.tolist():
    cur_household_data = cur_household_data.rename(columns={'latitude':'h_lat','longitude':'h_lng'})

print('household properties updated')
print('this oblast size',cur_household_data.shape)

neighbor_household_data = pd.read_csv(HOUSEHOLD_DIR+'ukraine_neighbor_'+PLACE_NAME+'_'+str(STRUCT)+'_s2.csv')

min_date = pd.to_datetime('2022-02-24')
end_date = pd.to_datetime('2022-05-15')
simulated_refugee_df = pd.DataFrame(columns=['id','time','refugee','old_people','child','male','female'])
simulated_leaving_df = pd.DataFrame(columns=['id','time','leaving','old_people','child','male','female'])
timing_log_df = pd.DataFrame(columns=['step','remaining_household_agent','remaining_person_agent','conflict_events_now','network_nodes',
                                      'network_edges','attitude_time','pcb_time','subjective_norm_time','pre_time','post_time'])

temp_prefix = ''
if ablation_conflict_type!='None':
    OUTPUT_DIR = ABLATION_DIR
    temp_prefix = 'ablation_'
    
print(OUTPUT_DIR)
print(temp_prefix)

f = 0

start = time.time()
cur_checkpoint = 1000
print('combination_no',hyper_comb)

prev_temp_checkpoint = 0
last_saved_checkpoint = -1

peer_used = 0

DEL_COLUMNS = ['P(violence)','P(move)','random']

who_went_where = []
hid_displacement_df = []

impact events reassigned (34022, 10)
household properties updated
this oblast size (274572, 20)
/project/biocomplexity/UKR_forecast/migration_data/output_data/

combination_no 99999
CPU times: user 46.8 s, sys: 6.18 s, total: 53 s
Wall time: 53.3 s


In [8]:
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

4552628

In [9]:
neighbor_household_data

,hid_x,s2_cell,hid_y
0,14,CellId: 40d7da7400000000,14
1,14,CellId: 40d7da7400000000,3262
2,14,CellId: 40d7da7400000000,6249
3,14,CellId: 40d7da7400000000,11829
4,14,CellId: 40d7da7400000000,71572
...,...,...,...
81699123,18815215,CellId: 40d7746400000000,18815215
81699124,18844752,CellId: 40da278c00000000,18844752
81699125,18844752,CellId: 40da278c00000000,18933988
81699126,18933988,CellId: 40da278c00000000,18844752


In [10]:
#########################################5
for i in range(0,300):
    print('at step',i)
    preprocess_start = time.time()
    
    prev_temp_checkpoint = prev_temp_checkpoint + 1
    
    max_date = min_date + pd.DateOffset(days=1)
    
    lookahead_date_1 = min_date - pd.DateOffset(days=lookbefore_days_left)
    lookahead_date_2 = min_date - pd.DateOffset(days=lookbefore_days_right)
    
    #if(f==0 and (min_date not in refugee_data['time'].tolist())):
    #    min_date = max_date
    #    continue
    
    if(f==1 and min_date > end_date):
        break
    ##################
    print(min_date)
    if(f!=0):
        cur_household_data = pd.read_csv(TEMPORARY_DIR+'last_saved_household_data_'+str(temp_prefix)+str(PLACE_NAME)+'_'+str(hyper_comb)+'.csv')
        #print(cur_household_data.info())
        #break
        cur_household_data = cur_household_data[cur_household_data.moves==0]
        
    ##################
    
    #print('number of households',cur_household_data.shape)
    
    #print('persons remaining',person_df.shape)
    cur_impact_data = impact_data[(impact_data.time>=lookahead_date_1) & (impact_data.time<=lookahead_date_2)]
    #cur_impact_data = cur_conflict_data[cur_conflict_columns]
    preprocess_end = time.time()
    
    attitude_start = time.time()
    cur_impact_data = cur_impact_data.rename(columns={'latitude':'impact_lat','longitude':'impact_lng'})
    cur_impact_data['cur_time'] = min_date
    cur_impact_data['time_diff_to_event'] = (cur_impact_data['cur_time'] - cur_impact_data['time']) / np.timedelta64(1,'D')
    cur_impact_data['impact_intensity'] = cur_impact_data['event_weight']*cur_impact_data['event_intensity']*EVENT_WEIGHT_SCALAR
    cur_impact_data['impact_intensity'].replace(to_replace = 0, value = EPS, inplace=True)
    
    if(cur_impact_data.shape[0]==0):
        new_row = {'id':PLACE_NAME,'time':min_date,'refugee':0,'old_people':0,'child':0,'male':0,'female':0}
        new_row_2 = {'id':PLACE_NAME,'time':min_date,'leaving':0,'old_people':0,'child':0,'male':0,'female':0}
        min_date = max_date
        simulated_refugee_df = simulated_refugee_df.append(new_row,ignore_index=True)
        simulated_leaving_df = simulated_leaving_df.append(new_row_2,ignore_index=True)
        continue
    
    if USE_NEIGHBOR==1:
        cur_household_data['key'] = 1
        cur_impact_data['key'] = 1
        impact_in_homes = cur_impact_data.merge(cur_household_data,on='key',how='inner')
    else:
        impact_in_homes = cur_impact_data.merge(cur_household_data,on='matching_place_id',how='inner')
    
    impacted_household_list = impact_in_homes.hid.unique().tolist()
    
    household_not_impacted = cur_household_data[~cur_household_data.hid.isin(impacted_household_list)]
    
    #print('number of impacted households',len(impact_in_homes.hid.unique().tolist()))
    #print('number of non impacted households',household_not_impacted.shape)
    
    if impact_in_homes.shape[0]==0:
        new_row = {'id':PLACE_NAME,'time':min_date,'refugee':0,'old_people':0,'child':0,'male':0,'female':0}
        new_row_2 = {'id':PLACE_NAME,'time':min_date,'leaving':0,'old_people':0,'child':0,'male':0,'female':0}
        min_date = max_date
        simulated_refugee_df = simulated_refugee_df.append(new_row,ignore_index=True)
        simulated_leaving_df = simulated_leaving_df.append(new_row_2,ignore_index=True)
        continue
    #print(ukraine_conflict_in_homes['fatalities'].describe())
    impact_in_homes['dis_conflict_home'] = haversine(impact_in_homes['h_lng'],impact_in_homes['h_lat'],impact_in_homes['impact_lng'],impact_in_homes['impact_lat'])
    impact_in_homes['prob_conflict'] = impact_in_homes['prob_conflict'].apply(lambda x: memory_decay(x))
    impact_in_homes['prob_conflict'] = impact_in_homes['prob_conflict'] + impact_in_homes.apply(lambda x: prob_conflict(x['impact_intensity'],x['dis_conflict_home'],x['time_diff_to_event']),axis=1)
    
    cur_household_data = cur_household_data.drop(columns='prob_conflict')
    home_conflict_df = impact_in_homes.groupby(['hid'])['prob_conflict'].sum().reset_index()
    home_conflict_df['P(violence)'] = home_conflict_df.prob_conflict.apply(aggregated_prob_conflict)
    home_conflict_df = home_conflict_df.merge(cur_household_data,on='hid',how='inner')
    #print('number of households with p(violence)',len(home_conflict_df.hid.unique().tolist()),home_conflict_df.shape)
    attitude_end = time.time()
    
    pcb_start = time.time()
    home_conflict_df['P(move)'] = home_conflict_df['P(violence)']*home_conflict_df['P(move|violence)']
    home_conflict_df['random'] = np.random.random(home_conflict_df.shape[0])
    home_conflict_df['moves'] = home_conflict_df.apply(lambda x: bernoulli(x['random'],x['P(move)']),axis=1)
    #print('number of households moving or not moving',home_conflict_df.shape)
    
    

    home_conflict_df = home_conflict_df.drop(columns=DEL_COLUMNS)
    
    #print('cur_save_household_size',home_conflict_df.shape)
    #print('saving columns',home_conflict_df.columns.tolist())
    print('###########################')
    pcb_end = time.time()
    
    subjective_norm_start = time.time()
    temp_households = pd.concat([home_conflict_df,household_not_impacted])
    nodes = temp_households.shape[0]
    
    #border_cross_low = 1 if peer_used<USE_PEER_EFFECT else 0
    phase = 1 
    #if(peer_used<min(30,USE_PEER_EFFECT)):
    #    border_cross_low = 2
    if(peer_used<PHASE_SHIFT):
        phase = 0
    
    if APPLY_PEER==1 and peer_used<USE_PEER_EFFECT:
        temp_households = refine_through_peer_effect(temp_households,phase)
        peer_used = peer_used + 1
    
    temp_households['move_type_random'] = np.random.random(temp_households.shape[0])
    temp_households['move_type'] = temp_households.apply(lambda x: bernoulli_border(x['move_type_random'],x['moves'],phase),axis=1)
    temp_households = temp_households.drop(columns=['move_type_random'])
    subjective_norm_end = time.time()
    
    post_process_and_save_start = time.time()
    new_row = {'id':PLACE_NAME,'time':min_date,'refugee':temp_households[temp_households.move_type==2]['hh_size'].sum(),
               'old_people':temp_households[temp_households.move_type==2]['OLD_PERSON'].sum(),'child':temp_households[temp_households.move_type==2]['CHILD'].sum(),
                'male':temp_households[temp_households.move_type==2]['ADULT_MALE'].sum(),'female':temp_households[temp_households.move_type==2]['ADULT_FEMALE'].sum()}
    #print(new_row)
    simulated_refugee_df = simulated_refugee_df.append(new_row,ignore_index=True)
    
    new_row_2 = {'id':PLACE_NAME,'time':min_date,'leaving':temp_households[temp_households.moves==1]['hh_size'].sum(),
               'old_people':temp_households[temp_households.moves==1]['OLD_PERSON'].sum(),'child':temp_households[temp_households.moves==1]['CHILD'].sum(),
                'male':temp_households[temp_households.moves==1]['ADULT_MALE'].sum(),'female':temp_households[temp_households.moves==1]['ADULT_FEMALE'].sum()}
    #print(new_row)
    simulated_leaving_df = simulated_leaving_df.append(new_row_2,ignore_index=True)
    
    temp_households['move_date'] = str(min_date)
    hid_displacement_df.append(temp_households[temp_households.move_type!=0])
    temp_households = temp_households.drop(columns=['move_date'])
    
    curtime = time.time()
    if((curtime-start)>=cur_checkpoint):
        print('checkpoint for ',str(PLACE_NAME))
        simulated_refugee_df.to_csv(OUTPUT_DIR+'mim_result_'+str(PLACE_NAME)+'_'+str(hyper_comb).zfill(5)+'.csv',index=False)
        simulated_leaving_df.to_csv(OUTPUT_DIR+'mim_result_leaving_'+str(PLACE_NAME)+'_'+str(hyper_comb).zfill(5)+'.csv',index=False)
        start = curtime
    
    temp_households.to_csv(TEMPORARY_DIR+'last_saved_household_data_'+str(temp_prefix)+str(PLACE_NAME)+'_'+str(hyper_comb)+'.csv',index=False)
    
    print('intention module done')
    
    ##apply destination module from here
    
    
    
    moving_households_df = temp_households[temp_households.move_type==2].reset_index()
    #print(moving_households_df.shape, 'people moved',end=' ')
    dest_prob_distribution_df = pd.read_csv(OUTPUT_DIR+'destination_probability_distribution.csv')
    
    if(str(min_date) not in dest_prob_distribution_df.columns.tolist()):
        #print(str(min_date))
        cur_dest_prob_df = dest_prob_distribution_df[['ISO','2022-03-01 00:00:00']]
        weight_col = '2022-03-01 00:00:00'
        #print(cur_dest_prob_df.shape)
    else:
        #print(str(min_date))
        cur_dest_prob_df = dest_prob_distribution_df[['ISO',str(min_date)]]
        weight_col = str(min_date)
    #print('projected')
    
    dest_sampled = cur_dest_prob_df.sample(moving_households_df.shape[0],replace=True,weights=cur_dest_prob_df[weight_col]).reset_index()
    #print(dest_sampled)
    #print(dest_sampled.shape,'sampled')
    #print(dest_sampled.ISO.value_counts())
    
    #break
    moving_households_df['destination'] = dest_sampled['ISO']
    moving_households_df['moved_date'] = str(min_date)
    
    who_went_where.append(moving_households_df)
    
    #print(moving_households_df['destination'].value_counts())
    
    ##destination  module done
    
    print('destination module done')
    
    last_saved_checkpoint = prev_temp_checkpoint
    min_date = max_date
    f = 1
    
    post_process_and_save_end = time.time()
    
    timing_row = {'step':i,'remaining_household_agent':cur_household_data.shape[0],'remaining_person_agent':cur_household_data['hh_size'].sum(),'conflict_events_now':cur_impact_data.shape[0],
                  'network_nodes':nodes,'network_edges':nodes*nodes,'attitude_time':attitude_end-attitude_start,'pcb_time':pcb_end-pcb_start,
                  'subjective_norm_time':subjective_norm_end-subjective_norm_start,'pre_time':preprocess_end-preprocess_start,'post_time':post_process_and_save_end-post_process_and_save_start}
    timing_log_df = timing_log_df.append(timing_row,ignore_index=True)
    


at step 0
2022-02-24 00:00:00
at step 1
2022-02-25 00:00:00
at step 2
2022-02-26 00:00:00
at step 3
2022-02-27 00:00:00
###########################
['hid_x', 's2_cell', 'hid_y', 'moves_y']
network creation through node merging by s2 takes 45.58691167831421 seconds
neighbor countint through groupby takes 2.2735912799835205 seconds
applying threshold function takes 2.3629395961761475 seconds
(274572, 4) (274572, 20)
intention module done
destination module done
at step 4
2022-02-28 00:00:00
###########################
['hid_x', 's2_cell', 'hid_y', 'moves_y']
network creation through node merging by s2 takes 45.16545224189758 seconds
neighbor countint through groupby takes 2.3983895778656006 seconds
applying threshold function takes 2.343599319458008 seconds
(274266, 4) (274266, 20)
intention module done
destination module done
at step 5
2022-03-01 00:00:00
###########################
['hid_x', 's2_cell', 'hid_y', 'moves_y']
network creation through node merging by s2 takes 45.06691384315

In [11]:
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

14506396

In [7]:
%%time
cur_household_data = pd.read_csv(HOUSEHOLD_DIR+'ukraine_household_data_ADM2_HDX.csv')
cur_household_data = cur_household_data[cur_household_data.matching_place_id==PLACE_NAME]
cur_household_data.shape

CPU times: user 21 s, sys: 1.59 s, total: 22.6 s
Wall time: 22.7 s


(274572, 15)

In [10]:
%%time
cur_struct = 13
cur_household_data['s2_cell'] = cur_household_data.apply(lambda x: getl13(x['latitude'],x['longitude'],cur_struct),axis=1)
temp_household = cur_household_data[['hid','s2_cell']]
neighbor_household = temp_household.merge(temp_household,on='s2_cell',how='inner')
neighbor_household.to_csv(HOUSEHOLD_DIR+'ukraine_neighbor_'+PLACE_NAME+'_'+str(cur_struct)+'_s2.csv',index=False)

CPU times: user 16min 17s, sys: 2.82 s, total: 16min 20s
Wall time: 16min 27s


In [3]:
import pandas as pd
from file_paths_and_consts import *
impact_data = pd.read_csv(IMPACT_DIR+'ukraine_conflict_data_ADM2_HDX_buffer_'+str(5)+'_km.csv')
cur_household_data = pd.read_csv(HOUSEHOLD_DIR+'ukraine_household_data_ADM2_HDX.csv')

In [5]:
impact_data['matching_place_id'].unique()

array(['Umanskyi', 'Vasylivskyi', 'Luhanskyi', 'Melitopolskyi',
       'Donetskyi', 'Pokrovskyi', 'Volnovaskyi', 'Kalmiuskyi',
       'Svativskyi', 'Bakhmutskyi', 'Sievierodonetskyi', 'Kryvorizkyi',
       'Kharkivskyi', 'Bohodukhivskyi', 'Nikopolskyi', 'Kramatorskyi',
       'Mariupolskyi', 'Berdianskyi', 'Zaporizkyi', 'Polohivskyi',
       'Beryslavskyi', 'Iziumskyi', 'Mykolaivskyi', 'Khersonskyi',
       'Horlivskyi', 'Kupianskyi', 'Chuhuivskyi', 'Bashtanskyi',
       'Dniprovskyi', 'Okhtyrskyi', 'Alchevskyi', 'Odeskyi', 'Sumskyi',
       'Bakhchysaraiskyi', 'Dzhankoiskyi', 'Kakhovskyi', 'Synelnykivskyi',
       'Novhorod-Siverskyi', 'Lozivskyi', 'Yevpatoriiskyi',
       'Krasnoperekopskyi', 'Shostkynskyi', 'Henicheskyi', 'Skadovskyi',
       'Koriukivskyi', 'Chernihivskyi', 'Lvivskyi', 'Konotopskyi', 'Kyiv',
       'Pavlohradskyi', 'Chernivetskyi', 'Voznesenskyi', 'Kremenchutskyi',
       'Pervomaiskyi', 'Simferopolskyi', 'Yaltynskyi', 'Dovzhanskyi',
       'Nizhynskyi', 'Romenskyi

In [6]:
cur_household_data['matching_place_id'].unique().tolist()

['Kalmiuskyi',
 'Synelnykivskyi',
 'Fastivskyi',
 'Khustskyi',
 'Berezivskyi',
 'Beryslavskyi',
 'Dubenskyi',
 'Novomoskovskyi',
 'Shepetivskyi',
 'Chernihivskyi',
 'Chortkivskyi',
 'Cherkaskyi',
 'Melitopolskyi',
 'Poltavskyi',
 'Zaporizkyi',
 'Volnovaskyi',
 'Nizhynskyi',
 'Kremenchutskyi',
 'Prylutskyi',
 'Volodymyrskyi',
 'Henicheskyi',
 'Starobilskyi',
 'Kovelskyi',
 'Sievierodonetskyi',
 'Kropyvnytskyi',
 'Kamianskyi',
 'Khersonskyi',
 'Zviahelskyi',
 'Chernivetskyi',
 'Khmelnytskyi',
 'Bilotserkivskyi',
 'Yavorivskyi',
 'Bolhradskyi',
 'Polohivskyi',
 'Odeskyi',
 'Sumskyi',
 'Berdianskyi',
 'Sarnenskyi',
 'Varaskyi',
 'Vyzhnytskyi',
 'Rakhivskyi',
 'Zhmerynskyi',
 'Dovzhanskyi',
 'Kamin-Kashyrskyi',
 'Podilskyi',
 'Haisynskyi',
 'Skadovskyi',
 'Rivnenskyi',
 'Sambirskyi',
 'Kupianskyi',
 'Mariupolskyi',
 'Kryvorizkyi',
 'Bashtanskyi',
 'Myrhorodskyi',
 'Bakhchysaraiskyi',
 'Voznesenskyi',
 'Konotopskyi',
 'Kamianets-Podilskyi',
 'Chervonohradskyi',
 'Lubenskyi',
 'Izmailskyi',
 